In [15]:
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
import logging

app = FastAPI()

class TranscribeRequest(BaseModel):
    video_url: str
    api_key: str

def verify_api_key(request: TranscribeRequest):
    if request.api_key != "test":  # Replace with your actual API key
        raise HTTPException(status_code=403, detail="Invalid API Key")

@app.get("/")
async def read_root():
    return {"message": "FastAPI is running"}

@app.post("/transcribe")
async def get_videoid(request: TranscribeRequest, verified: None = Depends(verify_api_key)):
    video_id = request.video_url.split("v=")[1]
    return await root(video_id)

async def root(video_id: str):
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    logging.info(f"Transcript for video ID {video_id}: {transcript}")
    formatter = TextFormatter()
    text_formatted = formatter.format_transcript(transcript)
    return {"transcript": text_formatted}

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from pyairtable import Api
api = Api(os.environ['AIRTABLE_TOKEN'])
table = api.table('appyl62z9rlWyAxMh', 'tblpc6rPUfZTlEcyE')

In [6]:
def get_active_records_with_api_key(api_key_value):
    try:
        # Fetch all records from the table
        all_records = table.all()
        
        # Filter records where 'Active' is True and 'x-api-key' matches the specific value
        filtered_records = [
            record for record in all_records
            if record['fields'].get('Active') and record['fields'].get('x-api-key') == api_key_value
        ]
        
        return filtered_records

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
specific_api_key = '5iqcnhqx4snag8fxswa4q7jjc0xkgc'
filtered_records = get_active_records_with_api_key(specific_api_key)

# Print the filtered records
for record in filtered_records:
    print(record)

{'id': 'recOiu9VOdy8qGv1C', 'createdTime': '2024-07-26T07:27:37.000Z', 'fields': {'Name': 'Dirk Moreno', 'x-api-key': '5iqcnhqx4snag8fxswa4q7jjc0xkgc', 'Active': True}}


In [7]:
from simple_salesforce import Salesforce

# Authenticate to Salesforce
sf = Salesforce(
    username='dirk@morenolabs.sandbox',
    password='MakeTradingGreatAgain73!',
    security_token='OSmhaZdm1XbgWTdextnZZCNVm'
)

# Define the new ticket (case) details
new_ticket = {
    'Subject': 'New Support Ticket',
    'Description': 'Description of the support issue',
    'Status': 'New',
    'Priority': 'High',
    'Origin': 'Web'
}

# Create the ticket
result = sf.Case.create(new_ticket)

print("Ticket created with ID:", result['id'])


Ticket created with ID: 500Qy00000GssILIAZ


In [9]:
new_customer = {
    'FirstName': 'John',
    'LastName': 'Doe',
    'Email': 'john.doe@example.com',
    'Phone': '123-456-7890'
}

# Create the customer
result = sf.Contact.create(new_customer)

print("Customer created with ID:", result['id'])

Customer created with ID: 003Qy000006VmXxIAK


In [13]:
# Step 1: Create a new account (if needed)
new_account = {
    'Name': 'Doe Inc.'
}
account_result = sf.Account.create(new_account)
account_id = account_result['id']
print("Account created with ID:", account_id)

# Step 2: Attempt to create a new contact
new_contact = {
    'FirstName': 'John',
    'LastName': 'Doe1',
    'Email': 'john.doe@example.com',
    'Phone': '123-456-7890',
    'AccountId': account_id
}

try:
    contact_result = sf.Contact.create(new_contact)
    contact_id = contact_result['id']
    print("Contact created with ID:", contact_id)
except SalesforceMalformedRequest as e:
    error_content = e.content
    if error_content[0]['errorCode'] == 'DUPLICATES_DETECTED':
        # Extract the existing contact ID from the error message
        existing_contact_id = error_content[0]['duplicateResult']['matchResults'][0]['matchRecords'][0]['record']['Id']
        print("Duplicate detected. Using existing contact with ID:", existing_contact_id)
        contact_id = existing_contact_id

# Step 3: Generate a new ticket (case) for the contact
new_ticket = {
    'Subject': 'Support Needed',
    'Description': 'Customer is experiencing issues with the product.',
    'Status': 'New',
    'Priority': 'High',
    'Origin': 'Web',
    'ContactId': contact_id,  # Associate the ticket with the created or existing contact
    'AccountId': account_id  # Optionally associate the ticket with the account
}

# Create the ticket
ticket_result = sf.Case.create(new_ticket)
print("Ticket created with ID:", ticket_result['id'])

Account created with ID: 001Qy00000Q3egJIAR
Contact created with ID: 003Qy000006VnozIAC
Ticket created with ID: 500Qy00000GsvOHIAZ
